<a href="https://colab.research.google.com/github/z1w0k/CLIP_TEST/blob/master/clip_2_part.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from torchvision.datasets import Imagenette
import numpy as np
from torch.utils.data import DataLoader
from transformers import CLIPProcessor, CLIPModel
import torch
import tqdm
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

CLIPModel LOAD REPORT from: openai/clip-vit-base-patch32
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
vision_model.embeddings.position_ids | UNEXPECTED |  | 
text_model.embeddings.position_ids   | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

The image processor of type `CLIPImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [ ]:
dataset = Imagenette(root = './data', split = 'val', download = True)
class_names = dataset.classes

100%|██████████| 1.56G/1.56G [00:27<00:00, 56.1MB/s]


In [ ]:
prompts = [f"a photo of {name}" for name in class_names]

In [ ]:
y_true = []
y_pred = []

text_inputs = processor(text = prompts, return_tensors = 'pt', padding = True)
text_features = model.get_text_features(**text_inputs).pooler_output
text_features = text_features / text_features.norm(dim = 1, keepdim = True)

for image, label in tqdm.tqdm(dataset, desc = 'Total'):
  image_inputs = processor(images = image, return_tensors = 'pt', padding = True)
  image_features = model.get_image_features(**image_inputs).pooler_output
  image_features = image_features / image_features.norm(dim = 1, keepdim = True)

  similarity = image_features @ text_features.T
  pred = similarity.argmax(dim = 1)

  y_pred.append(pred)
  y_true.append(label)

accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average = 'micro')

print(f"\nAccuracy = {100 * accuracy:.2f}%")
print(f"F1 = {f1:.2f}")

Total: 100%|██████████| 3925/3925 [18:15<00:00,  3.58it/s]



Accuracy = 98.01%
F1 = 0.98
